
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# 言語モデル

:label: `sec_language-model`

 :numref: `sec_text-sequence`では、テキスト シーケンスをトークンにマッピングする方法がわかります。これらのトークンは、単語や文字などの個別の観察のシーケンスとして見ることができます。長さ $T$ のテキスト シーケンス内のトークンが、順に $x_1、x_2、\ldots、x_T$ であると仮定します。*言語モデル*の目標は、シーケンス全体の結合確率を推定することです。

 $$P(x_1, x_2, \ldots, x_T),$$

ここで、:numref: `sec_sequence`の統計ツールを適用できます。

言語モデルは非常に便利です。たとえば、理想的な言語モデルでは、一度に 1 つのトークン $x_t \sim P(x_t \mid x_{t-1}, \ldots, x_1)$ を描画するだけで、自然テキストを単独で生成できます。タイプライターを使用する猿とはまったく異なり、そのようなモデルから生成されるすべてのテキストは、自然言語 (たとえば、英語のテキスト) として渡されます。さらに、前のダイアログの断片に基づいてテキストを条件付けするだけで、意味のあるダイアログを生成するのに十分です。文法的に意味のあるコンテンツを生成するだけではなく、テキスト*を理解する*必要があるため、そのようなシステムの設計にはまだかなり遠いのは明らかです。

それにもかかわらず、言語モデルは、その限られた形式であっても非常に役立ちます。たとえば、「音声を認識する」と「素敵なビーチを破壊する」というフレーズは非常に似ています。これにより音声認識に曖昧さが生じる可能性がありますが、これは 2 番目の翻訳を奇抜なものとして拒否する言語モデルを通じて簡単に解決できます。同様に、文書要約アルゴリズムでは、「犬が人を噛む」の方が「人が犬を噛む」よりもはるかに頻繁であること、または「おばあちゃんを食べたい」はかなり不安な発言であるのに対し、「私は食べたい、おばあちゃん」の方がずっと優しいです。


In [1]:
import torch
from d2l import torch as d2l


## 学習言語モデル

明らかな問題は、ドキュメント、さらにはトークンのシーケンスをどのようにモデル化すべきかということです。テキストデータを単語レベルでトークン化するとします。基本的な確率ルールを適用することから始めましょう。

 $$P(x_1, x_2, \ldots, x_T) = \prod_{t=1}^TP(x_t \mid x_1, \ldots, x_{t-1}).$$

たとえば、テキスト シーケンスに 4 つの単語が含まれる確率は次のように与えられます。

 $$\begin{aligned}&amp;P(\text{deep}, \text{learning}, \text{is}, \text{fun}) \ =&amp;P(\text{deep}) P(\text{learning} \mid \text{深い}) P(\text{は} \mid \text{深い}, \text{学習}) P(\text{楽しい} \mid \text{深い}, \text{学習}, \text{is}).\end{aligned}$$

### マルコフ モデルと $n$-gram

 :numref: `sec_sequence`のシーケンス モデル分析のうち、マルコフ モデルを言語モデリングに適用してみましょう。 $P(x_{t+1} \mid x_t, \ldots, x_1) = P(x_{t+1} \mid x_t)$ の場合、数列にわたる分布は一次マルコフ特性を満たします。より高い順序はより長い依存関係に対応します。これにより、シーケンスのモデル化に適用できるいくつかの近似が得られます。

 $$ \begin{aligned} P(x_1, x_2, x_3, x_4) &amp;= P(x_1) P(x_2) P(x_3) P(x_4),\ P(x_1, x_2, x_3, x_4) &amp;= P (x_1) P(x_2 \mid x_1) P(x_3 \mid x_2) P(x_4 \mid x_3),\ P(x_1, x_2, x_3, x_4) &amp;= P(x_1) P(x_2 \mid x_1) P (x_3 \mid x_1, x_2) P(x_4 \mid x_2, x_3)。 \end{整列} $$

 1 つ、2 つ、および 3 つの変数を含む確率式は、通常、それぞれ*unigram* 、 *bigram* 、および*trigram*モデルと呼ばれます。言語モデルを計算するには、前のいくつかの単語を考慮して、単語の確率と単語の条件付き確率を計算する必要があります。このような確率は言語モデルのパラメーターであることに注意してください。

### 単語の頻度

ここでは、トレーニング データセットが、すべての Wikipedia エントリ、[プロジェクト グーテンベルク](https://en.wikipedia.org/wiki/Project_Gutenberg)、Web に投稿されたすべてのテキストなどの大規模なテキスト コーパスであると仮定します。単語の確率は、トレーニング データセット内の特定の単語の相対的な単語頻度から計算できます。たとえば、推定 $\hat{P}(\text{deep})$ は、単語「deep」で始まる文の確率として計算できます。若干精度が劣るアプローチとしては、「deep」という単語の出現をすべて数えて、それをコーパス内の単語の総数で割ることです。これは、特に頻繁に使用される単語の場合、かなりうまく機能します。次に、推定を試みることができます

$$\hat{P}(\text{学習} \mid \text{ディープ}) = \frac{n(\text{ディープ, ラーニング})}{n(\text{ディープ})},$$

ここで、$n(x)$ と $n(x, x&#39;)$ は、それぞれシングルトンと連続した単語のペアの出現数です。残念ながら、「深層学習」の発生頻度ははるかに低いため、単語のペアの確率を推定することは多少難しくなります。特に、一部の珍しい単語の組み合わせでは、正確な推定値を取得するのに十分な単語の出現を見つけるのが難しい場合があります。 :numref: `subsec_natural-lang-stat`の経験的結果が示唆しているように、3 つの単語の組み合わせ以降では状況は悪化します。データセットにはおそらく見られない、もっともらしい 3 単語の組み合わせが多数存在します。このような単語の組み合わせにゼロ以外のカウントを割り当てる何らかの解決策を提供しない限り、言語モデルでそれらを使用することはできません。データセットが小さい場合、または単語が非常にまれな場合、単語が 1 つも見つからない可能性があります。

### ラプラス平滑化

一般的な戦略は、何らかの形式の*ラプラス平滑化*を実行することです。解決策は、すべてのカウントに小さな定数を追加することです。 $n$ はトレーニング セット内の単語の総数、$m$ は一意の単語の数を示します。このソリューションは、シングルトンに役立ちます。

 $$\begin{aligned} \hat{P}(x) &amp; = \frac{n(x) + \epsilon_1/m}{n + \epsilon_1}, \ \hat{P}(x&#39; \mid x) &amp; = \frac{n(x, x&#39;) + \epsilon_2 \hat{P}(x&#39;)}{n(x) + \epsilon_2}, \ \hat{P}(x&#39;&#39; \mid x,x &#39;) &amp; = \frac{n(x, x&#39;,x&#39;&#39;) + \epsilon_3 \hat{P}(x&#39;&#39;)}{n(x, x&#39;) + \epsilon_3}。 \end{整列}$$

ここで、$\epsilon_1、\epsilon_2$、および $\epsilon_3$ はハイパーパラメーターです。 $\epsilon_1$ を例に挙げます。$\epsilon_1 = 0$ の場合、平滑化は適用されません。 $\epsilon_1$ が正の無限大に近づくと、$\hat{P}(x)$ は一様確率 $1/m$ に近づきます。上記は、他の技術で達成できるもののかなり原始的な変形です (cite: `Wood.Gasthaus.Archambeau.ea.2011` )。

残念ながら、このようなモデルは、次の理由により、すぐに扱いにくくなります。まず、 :numref: `subsec_natural-lang-stat`で説明したように、多くの $n$-gram は非常にまれに発生するため、ラプラス平滑化は言語モデリングにはかなり不向きです。次に、すべてのカウントを保存する必要があります。第三に、これは言葉の意味を完全に無視しています。たとえば、「猫」と「猫」は関連するコンテキストで出現する必要があります。このようなモデルを追加のコンテキストに合わせて調整することは非常に困難ですが、深層学習ベースの言語モデルはこれを考慮するのに適しています。最後に、長い単語シーケンスはほぼ確実に新規であるため、以前に見た単語シーケンスの頻度を単純にカウントするモデルは、そこではパフォーマンスが低下するはずです。したがって、この章の残りの部分では、言語モデリングにニューラル ネットワークを使用することに焦点を当てます。

## 困惑

:label: `subsec_perplexity`

次に、言語モデルの品質を測定する方法について説明します。これは、後続のセクションでモデルを評価するために使用されます。 1 つの方法は、テキストがどれほど驚くべきかを確認することです。優れた言語モデルは、次に何が起こるかを高精度のトークンで予測できます。さまざまな言語モデルによって提案されている、「雨が降っています」というフレーズの次の続きを考えてみましょう。
1.  "外は雨が降っています"
1.  「バナナの木に雨が降っている」
1.  「雨が降っています。kcj pwepoiut」

品質の点では、例 1 が明らかに最高です。言葉は理にかなっていて、論理的に一貫しています。意味的にどの単語が後に続くかを正確に反映していない可能性がありますが (「サンフランシスコで」と「冬で」は完全に合理的な拡張です)、モデルはどの種類の単語が後に続くかを捕捉できます。例 2 は、意味のない拡張を生成することにより、かなり悪化しています。それにもかかわらず、少なくともモデルは単語の綴り方と単語間のある程度の相関関係を学習しました。最後の例 3 は、データを適切に適合させていない、トレーニングが不十分なモデルを示しています。

シーケンスの尤度を計算することでモデルの品質を測定することもできます。残念ながら、これは理解しにくく、比較するのも難しい数字です。結局のところ、長いシーケンスよりも短いシーケンスが発生する可能性がはるかに高いため、トルストイの大作*『戦争と平和』*に基づいてモデルを評価すると、必然的に、たとえばサンテグジュペリの中編小説*『星の王子さま』*に基づいてモデルを評価するよりもはるかに低い確率が生成されます。欠けているのは平均に相当するものです。

ここで情報理論が役に立ちます。ソフトマックス回帰 (:numref: `subsec_info_theory_basics` ) を導入したときに、エントロピー、サプライズ、クロスエントロピーを定義しました。テキストを圧縮したい場合は、現在のトークンのセットを考慮して次のトークンを予測することについて尋ねることができます。より優れた言語モデルにより、次のトークンをより正確に予測できるようになります。したがって、シーケンスの圧縮に費やすビットを少なくできるはずです。したがって、シーケンスのすべての $n$ トークンの平均化されたクロスエントロピー損失によってそれを測定できます。

 $$\frac{1}{n} \sum_{t=1}^n -\log P(x_t \mid x_{t-1}, \ldots, x_1),$$ :eqlabel: `eq_avg_ce_for_lm`

ここで、$P$ は言語モデルによって与えられ、$x_t$ はシーケンスのタイム ステップ $t$ で観察された実際のトークンです。これにより、異なる長さのドキュメントのパフォーマンスが同等になります。歴史的な理由から、自然言語処理の科学者は*perplexity*と呼ばれる量を使用することを好みます。一言で言えば、これは :eqref: `eq_avg_ce_for_lm`の指数です。

 $$\exp\left(-\frac{1}{n} \sum_{t=1}^n \log P(x_t \mid x_{t-1}, \ldots, x_1)\right).$$

混乱は、次にどのトークンを選択するかを決定する際に存在する実際の選択肢の数の幾何平均として最もよく理解できます。いくつかのケースを見てみましょう。
- 最良のシナリオでは、モデルはターゲット トークンの確率を常に 1 として完全に推定します。この場合、モデルのパープレキシティは 1 です。
- 最悪のシナリオでは、モデルはターゲット トークンの確率を常に 0 と予測します。この状況では、混乱度は正の無限大になります。
- ベースラインでは、モデルは語彙の利用可能なすべてのトークンにわたる一様な分布を予測します。この場合、困惑度は語彙の一意のトークンの数に等しくなります。実際、シーケンスを圧縮せずに保存する場合、これがそれをエンコードするためにできる最善の方法です。したがって、これは、有用なモデルが超えなければならない自明ではない上限を提供します。

## パーティショニングシーケンス

:label: `subsec_partitioning-seqs`

ニューラル ネットワークを使用して言語モデルを設計し、パープレキシティを使用して、テキスト シーケンス内の現在のトークン セットを考慮して次のトークンを予測する際にモデルがどの程度優れているかを評価します。モデルを導入する前に、モデルが事前に定義された長さのシーケンスのミニバッチを一度に処理すると仮定しましょう。ここで問題は、[**入力シーケンスとターゲット シーケンスのミニバッチをランダムに読み取る**]方法です。

データセットが`corpus`内の一連の $T$ トークン インデックスの形式を取ると仮定します。これをサブシーケンスに分割します。各サブシーケンスには $n$ トークン (タイム ステップ) が含まれます。各エポックのデータセット全体の (ほぼ) すべてのトークンを反復処理し、考えられるすべての長さの $n$ サブシーケンスを取得するには、ランダム性を導入できます。より具体的には、各エポックの開始時に、最初の $d$ トークンを破棄します。ここで、$d\in [0,n)$ は均一にランダムにサンプリングされます。次に、残りのシーケンスは $m=\lfloor (Td)/n \rfloor$ サブシーケンスに分割されます。 $\mathbf x_t = [x_t, \ldots, x_{t+n-1}]$ によって、タイム ステップ $t$ でトークン $x_t$ から始まる長さ $n$ の部分列を表します。結果として得られる $m$ 分割された部分列は、$\mathbf x_d、\mathbf x_{d+n}、\ldots、\mathbf x_{d+n(m-1)} です。$ 各部分列は、への入力シーケンスとして使用されます。言語モデル。

言語モデリングの目的は、これまでに確認したトークンに基づいて次のトークンを予測することです。したがって、ターゲット (ラベル) は、1 トークン分シフトされた元のシーケンスになります。入力シーケンス $\mathbf x_t$ のターゲット シーケンスは、長さが $n$ の $\mathbf x_{t+1}$ です。 

![](../img/lang-model-data.svg) :label: `fig_lang_model_data`

 :numref: `fig_lang_model_data` $n=5$、$d=2$ として、5 組の入力シーケンスとターゲット シーケンスを取得する例を示しています。


In [2]:
@d2l.add_to_class(d2l.TimeMachine)  #@save
def __init__(self, batch_size, num_steps, num_train=10000, num_val=5000):
    super(d2l.TimeMachine, self).__init__()
    self.save_hyperparameters()
    corpus, self.vocab = self.build(self._download())
    array = torch.tensor([corpus[i:i+num_steps+1]
                        for i in range(len(corpus)-num_steps)])
    self.X, self.Y = array[:,:-1], array[:,1:]


言語モデルをトレーニングするには、ミニバッチ内の入力シーケンスとターゲット シーケンスのペアをランダムにサンプリングします。次のデータ ローダーは、毎回データセットからランダムにミニバッチを生成します。引数`batch_size`各ミニバッチ内のサブシーケンスの例の数を指定し、 `num_steps`はトークン内のサブシーケンスの長さです。


In [3]:
@d2l.add_to_class(d2l.TimeMachine)  #@save
def get_dataloader(self, train):
    idx = slice(0, self.num_train) if train else slice(
        self.num_train, self.num_train + self.num_val)
    return self.get_tensorloader([self.X, self.Y], train, idx)


以下でわかるように、ターゲット シーケンスのミニバッチは、入力シーケンスを 1 トークンだけシフトすることによって取得できます。


In [4]:
data = d2l.TimeMachine(batch_size=2, num_steps=10)
for X, Y in data.train_dataloader():
    print('X:', X, '\nY:', Y)
    break

X: tensor([[ 0,  5, 10, 14,  6, 15, 20, 10, 16, 15],
        [ 5, 10,  7,  7,  6, 19,  6, 15,  4,  6]]) 
Y: tensor([[ 5, 10, 14,  6, 15, 20, 10, 16, 15,  0],
        [10,  7,  7,  6, 19,  6, 15,  4,  6,  0]])



## 要約と考察

言語モデルは、テキスト シーケンスの同時確率を推定します。長いシーケンスの場合、$n$-gram は依存関係を切り詰めることにより便利なモデルを提供します。ただし、構造はたくさんありますが、ラプラス平滑化によって頻度の低い単語の組み合わせを効率的に処理するには頻度が十分ではありません。したがって、後続のセクションではニューラル言語モデリングに焦点を当てます。言語モデルをトレーニングするには、ミニバッチ内の入力シーケンスとターゲット シーケンスのペアをランダムにサンプリングします。トレーニング後、パープレキシティを使用して言語モデルの品質を測定します。

言語モデルは、データ サイズ、モデル サイズ、トレーニング コンピューティングの量を増やすことでスケールアップできます。大規模な言語モデルは、入力テキストの指示から出力テキストを予測することで、目的のタスクを実行できます。後で説明するように (例:numref: `sec_large-pretraining-transformers` )、現時点では、大規模な言語モデルがさまざまなタスクにわたる最先端のシステムの基礎を形成しています。

## 演習
1. トレーニング データセットに $100,000$ の単語があると仮定します。 4 グラムにはどのくらいの単語頻度と複数単語の隣接頻度を保存する必要がありますか?
1. 対話をどのようにモデル化しますか?
1. 長いシーケンスデータを読み取るために他にどのような方法が考えられますか?
1. 各エポックの開始時に最初のいくつかのトークンの一様乱数を破棄する方法を考えてみましょう。<ol><li>本当に文書上の配列全体に完全に均一な分布が得られるのでしょうか?
1. 物事をさらに均一にするためには何をする必要がありますか?



[ディスカッション](https://discuss.d2l.ai/t/118)
